In [1]:
path = '../Scripts/'
test_path = '../Data/test.csv'
train_path = '../Data/clean_data.csv'

In [2]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
sys.path.append(path)
import Data_cleaner
cleaner = Data_cleaner.data_cleaner()

In [3]:
# import the cleanded train data and test data
test = pd.read_csv(test_path)
train = pd.read_csv(train_path)

C:\Users\Akram 1\AppData\Local\Temp\ipykernel_21480\3385987091.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(train_path)


In [6]:
# lets check if there are any nan values in the test data
cleaner.percent_missing(test)
# lets use the same technique we used to clean the train data

0.0

In [5]:
#For the rest attribuite, its better to drop the column.
test = test.fillna('mean')

In [6]:
cleaner.percent_missing(test)

0.0

In [7]:
print(train.columns)

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday'],
      dtype='object')


In [8]:
import pandas as pd
import numpy as np

# Step 1: Apply one-hot encoding to categorical variables
train = pd.get_dummies(train, drop_first=True)  # drop_first to avoid multicollinearity
test = pd.get_dummies(test, drop_first=True)

# Step 2: Align the train and test datasets
train, test = train.align(test, join='left', axis=1, fill_value=0)

# Step 3: Feature Engineering
# Identify date columns (assuming columns start with 'Date_')
date_columns = [col for col in train.columns if col.startswith('Date_')]

# Create a 'Date' column from existing date columns
if date_columns:
    # Create a 'Date' column by converting the column names into actual datetime objects
    # Example: If the column is 'Date_2013-01-01', we can directly use it to create a datetime object
    # Create a list of dates by parsing the column names
    train['Date'] = pd.to_datetime(date_columns[0].replace('Date_', ''), format='%Y-%m-%d', errors='coerce')
    test['Date'] = pd.to_datetime(date_columns[0].replace('Date_', ''), format='%Y-%m-%d', errors='coerce')

    # Extract date features
    for dataset in [train, test]:
        dataset['Year'] = dataset['Date'].dt.year
        dataset['Month'] = dataset['Date'].dt.month
        dataset['Day'] = dataset['Date'].dt.day
        dataset['DayOfWeek'] = dataset['Date'].dt.dayofweek
        dataset['WeekOfYear'] = dataset['Date'].dt.isocalendar().week

# Step 4: Create Interaction Features
train['Customers_Promo'] = train['Customers'] * train['Promo']
test['Customers_Promo'] = test['Customers'] * test['Promo']

# Step 5: Log Transformation of Sales
if 'Sales' in train.columns:
    train['LogSales'] = np.log1p(train['Sales'])  # Log-transform sales
    y_train = train['LogSales']  # Target variable
    X_train = train.drop(['Sales', 'LogSales', 'Date'], axis=1)  # Features

    # Prepare test features (without the target)
    X_test = test.drop(['Sales', 'Date'], axis=1)  # Ensure to drop 'Sales' if it's present

# Split the data into prediction target 'y' ('salesprice), and attributes 'X'
y = train['Sales']
X = train.drop('Sales', axis=1)

# First lets split the training dataset into test and train and then use them to compare the two choosen models (XGBRegressor and RandomForestRegressor)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
modelR = RandomForestRegressor(n_estimators= 100, random_state=0)

modelR.fit(X_train, y_train)
prediction1 = modelR.predict(X_val)
Error = mean_absolute_error(y_val, prediction1)
print(Error)


In [9]:
# modelX = XGBRegressor(n_estimators = 200, learning_rate = 0.05, n_jobs=4)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Assuming `train` and `test` DataFrames are already defined

# Step 1: Apply one-hot encoding to categorical variables
train = pd.get_dummies(train, drop_first=True)  # drop_first to avoid multicollinearity
test = pd.get_dummies(test, drop_first=True)

# Step 2: Align the train and test datasets
train, test = train.align(test, join='left', axis=1, fill_value=0)

# Step 3: Feature Engineering
# Identify date columns (assuming columns start with 'Date_')
date_columns = [col for col in train.columns if col.startswith('Date_')]

# Create a 'Date' column from existing date columns if any
if date_columns:
    # Create a 'Date' column by converting the column names into actual datetime objects
    train['Date'] = pd.to_datetime(date_columns[0].replace('Date_', ''), format='%Y-%m-%d', errors='coerce')
    test['Date'] = pd.to_datetime(date_columns[0].replace('Date_', ''), format='%Y-%m-%d', errors='coerce')

    # Extract date features
    for dataset in [train, test]:
        dataset['Year'] = dataset['Date'].dt.year
        dataset['Month'] = dataset['Date'].dt.month
        dataset['Day'] = dataset['Date'].dt.day
        dataset['DayOfWeek'] = dataset['Date'].dt.dayofweek
        dataset['WeekOfYear'] = dataset['Date'].dt.isocalendar().week

# Step 4: Create Interaction Features
if 'Customers' in train.columns and 'Promo' in train.columns:
    train['Customers_Promo'] = train['Customers'] * train['Promo']
    test['Customers_Promo'] = test['Customers'] * test['Promo']

# Step 5: Log Transformation of Sales
if 'Sales' in train.columns:
    train['LogSales'] = np.log1p(train['Sales'])  # Log-transform sales
    y_train = train['LogSales']  # Target variable
    X_train = train.drop(['Sales', 'LogSales', 'Date'], axis=1)  # Features

    # Prepare test features (without the target)
    X_test = test.drop(['Sales', 'Date'], axis=1)  # Ensure to drop 'Sales' if it's present

    # Split the data into prediction target 'y' ('salesprice), and attributes 'X'
    y = train['Sales']
    X = train.drop('Sales', axis=1)

    # Split the training dataset into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

    # Initialize the RandomForestRegressor model
    modelR = RandomForestRegressor(n_estimators=100, random_state=0)

    # Fit the model
    modelR.fit(X_train, y_train)

    # Make predictions
    prediction1 = modelR.predict(X_val)

    # Calculate and print the error
    Error = mean_absolute_error(y_val, prediction1)
    print("Mean Absolute Error:", Error)
else:
    print("Sales column is missing in the training dataset.")


TypeError: Cannot cast DatetimeArray to dtype float32

In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

# Assuming you have already defined your X_train and y_train variables
# For example:
# X_train = train.drop(['Sales', 'LogSales', 'Date'], axis=1)
# y_train = train['LogSales']

# Define the function to compute the mean absolute error (MAE) for different estimators
def mae_score(estimator):
    model = RandomForestRegressor(n_estimators=estimator, random_state=0)
    val_score = -1 * cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')  # Use 5-fold CV
    return val_score.mean()

# List of n_estimators to try
n_estimators = [10, 50, 100, 200]
mae_scores = []

# Loop through the different n_estimators and compute the MAE
for n in n_estimators:
    score = mae_score(n)
    mae_scores.append(score)
    print(f"n_estimators: {n}, MAE: {score}")

# Plotting the results
plt.figure(figsize=(10, 6))
plt.plot(n_estimators, mae_scores, marker='o')
plt.title('MAE vs. Number of Estimators')
plt.xlabel('Number of Estimators')
plt.ylabel('Mean Absolute Error (MAE)')
plt.xscale('log')  # Use logarithmic scale for better visualization
plt.grid()
plt.show()

# Determine the best number of estimators
best_n_estimators = n_estimators[np.argmin(mae_scores)]
print(f"Best number of estimators: {best_n_estimators}")


In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
# Optionally split train into a train and validation set
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Initialize and train a Random Forest model
model = RandomForestRegressor(n_estimators=50, random_state=42)
model.fit(X_train_split, y_train_split)

# Predict and evaluate on validation set
y_val_pred = model.predict(X_val_split)
val_rmse = np.sqrt(mean_squared_error(y_val_split, y_val_pred))

print(f"Validation RMSE: {val_rmse}")

# Predict on the test set
y_test_pred = model.predict(X_test)

TypeError: Cannot cast DatetimeArray to dtype float32

In [12]:
# result = {}
# estimator = [50, 100, 200]

# for i in estimator:
#     result[i] = mae_score(i)


In [13]:

# %matplotlib inline
# plt.plot(list(result.keys()), list(result.values()))
# plt.show()

In [14]:
# Before fiting the data into the model, lets find the best number of estimators for both models
def mae_score(estimator):
    model2 = XGBRegressor(n_estimators = estimator, learning_rate = 0.05, n_jobs=4)
    val_score = -1 * cross_val_score(model2, X, y, cv=2, scoring='neg_mean_absolute_error')

    return val_score.mean()

In [15]:
# result2 = {}
# estimator = [25, 50, 100, 200]

# for i in estimator:
#     result2[i] = mae_score(i)

In [16]:

# %matplotlib inline
# plt.plot(list(result2.keys()), list(result2.values()))
# plt.show()

In [17]:
# so lets use 200 estimators for XGBoost and 100 estimators for RandomForest
modelX = XGBRegressor(n_estimators = 200, learning_rate = 0.05, n_jobs=4)
modelR = RandomForestRegressor(n_estimators= 100, random_state=0)


In [ ]:
modelR.fit(X_train, y_train)
prediction1 = modelR.predict(X_val)
Error = mean_absolute_error(y_val, prediction1)
print(Error)

In [ ]:
modelX.fit(X_train, y_train)
prediction2 = modelX.predict(X_val)
Error2 = mean_absolute_error(y_val, prediction2)
print(Error2)

In [20]:
common_cols = test.columns.intersection(X.columns)

# Drop the columns that are not in common
test = test[common_cols]
X = X[common_cols]

In [21]:
#Hence RGBoost has the upper hand, now lets train it using the whole data
model = XGBRegressor(n_estimators = 200, learning_rate = 0.05, n_jobs=4)
model.fit(X, y)
prediction = model.predict(test)

In [22]:
bool_columns = test.select_dtypes(include='bool').columns
test[bool_columns] = test[bool_columns].astype(int)


In [ ]:
bool_columns = X_train.select_dtypes(include='bool').columns
test[bool_columns] = X_train[bool_columns].astype(int)


In [ ]:
test.dtypes

In [ ]:
import shap

explainer = shap.Explainer(model, test)
shap_values = explainer(test)

# Summary plot
shap.summary_plot(shap_values, test)


In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values.values[0], test.iloc[0])


In [ ]:
shap_values_array = shap_values.values

shap.dependence_plot('LotArea', shap_values_array, test, feature_names=test.columns)
